# Base de Dados Whatsapp

Este caderno trabalha com os dados gravados pelo whatsapp em msgstore.db

In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np

import sqlite3

from wordcloud import WordCloud
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize, wordpunct_tokenize


In [ ]:
# Efetua download de pacotes da boblioteca nltk
nltk.download('stopwords')
nltk.download('punkt')

## Conexao com Whatsapp DB

In [ ]:
# Conexao com o arquivo do banco de dados SQLITE
con = sqlite3.connect("Dados_Celular/Database/msgstore.db")

In [ ]:
cur = con.cursor()

## Trazer todas as mensagens para um DataFrame

In [ ]:
res_mensagens = cur.execute('SELECT * FROM messages')
df_mensagens = pd.DataFrame(res_mensagens.fetchall())
df_mensagens.head()

## Contatos com quem a pessoa teve mais conversas

In [ ]:
# Identificando maiores conversas
res = cur.execute("""SELECT key_remote_jid
                            ,count(*) AS number_of_messages
                     FROM messages
                     GROUP BY key_remote_jid
                     ORDER BY number_of_messages DESC
                     LIMIT 15""" )
df_maiores_conversas = pd.DataFrame(res.fetchall())
df_maiores_conversas.columns=['contato','quantidade']
df_maiores_conversas

In [ ]:
df_maiores_conversas.plot.bar()

In [ ]:
# Query para trazer todas as conversas com um contato em um DataFrame
contato = 'XXXXXXXXXXXXX@s.whatsapp.net'

query_conversa = f"""
                    SELECT *
                    FROM messages
                    WHERE key_remote_jid == '{contato}' 
                 """
# Executa a query e retorna o resultado em um DataFrame
res_conversa = cur.execute(query_conversa)
df_conversa = pd.DataFrame(res_conversa.fetchall())

# Pega o nome das colunas e renomeia no DataFrame
lista_colunas = []
for i in range(len(res_conversa.description)):
    lista_colunas.append(res_conversa.description[i][0])
df_conversa.columns = lista_colunas
    
#df_conversa.columns=['contato','quantidade']
pd.set_option('display.max_columns', None)
df_conversa.head()


In [ ]:
# Imprime a Nuvem de Palavras
# Ajusta o dataframe e junta todo o conteudo do campo transcricao em uma unica string sem espaços duplos

serie = df_conversa['data']
string_palavras = ' '
string_palavras = serie.to_string()
string_palavras = ' '.join(string_palavras.split())
    
# Remove as stop words
stopwords = set(nltk.corpus.stopwords.words('portuguese'))
outras_palavras_retirar = ['None','kkk', 'kkkk', 'kk', 'vc', 'bem', 'vou', 'não', 'Não']
for p in outras_palavras_retirar:
    stopwords.add(p)

palavras_tokenized = nltk.word_tokenize(string_palavras, language='portuguese')

palavras_para_nuvem = ' '

for palavra in palavras_tokenized:
    if not(palavra in stopwords):
        palavras_para_nuvem = palavras_para_nuvem + ' ' + palavra
    
# Plota a nuvem de palavras e grava em um arquivo de imagem
x, y = np.ogrid[:300, :300]

mask = (x - 150) ** 2 + (y - 150) ** 2 > 130 ** 2
mask = 255 * mask.astype(int)

wc = WordCloud(background_color="white", repeat=True, mask=mask)

wc.generate(palavras_para_nuvem)
#wc.to_file("Saida/wordcloud/laudo_XXX_20XX_item_XX.png")
plt.imshow(wc) 

